# Interactive Dashboard
Query the trained model interactively: look up recommendations for any movie or user, explore the preference network, and trace movie journeys.

In [1]:
import sys
from pathlib import Path

ROOT = Path().resolve().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import numpy as np
import pandas as pd

MODELS    = ROOT / 'outputs' / 'models'
PROCESSED = ROOT / 'data' / 'processed'

movies         = pd.read_parquet(PROCESSED / 'movies_enriched.parquet')
journey_edges  = pd.read_csv(MODELS / 'journey_edges.csv')
z              = np.load(MODELS / 'svd_artifacts.npz')
item_factors   = z['item_factors'].astype(np.float32)
movie_ids      = z['movie_ids'].astype(np.int32)

print(f'Movies: {len(movies):,} | Journey edges: {len(journey_edges):,} | SVD items: {item_factors.shape}')

Movies: 500 | Journey edges: 10,000 | SVD items: (500, 10)


In [2]:
# Interactive movie search
QUERY = 'Godfather'   # <-- change this to any title
results = movies[movies['title'].str.contains(QUERY, case=False, na=False)][['movieId', 'title', 'primary_genre', 'year']]
print(results.to_string(index=False))

 movieId                          title primary_genre  year
     858          Godfather, The (1972)         Crime  1972
    1221 Godfather: Part II, The (1974)         Crime  1974


In [3]:
# Cosine similarity recommendations for a chosen movie
SOURCE_MOVIE_ID = 858   # The Godfather — change as needed
TOP_N = 10

idx_map = {int(mid): i for i, mid in enumerate(movie_ids)}
if SOURCE_MOVIE_ID in idx_map:
    idx    = idx_map[SOURCE_MOVIE_ID]
    source = item_factors[idx]
    dots   = item_factors @ source
    norms  = np.linalg.norm(item_factors, axis=1) * np.linalg.norm(source)
    sims   = np.where(norms > 0, dots / norms, 0.0)
    sims[idx] = -1
    top    = np.argsort(sims)[::-1][:TOP_N]
    recs   = pd.DataFrame({'movieId': movie_ids[top], 'similarity': sims[top]})
    recs   = recs.merge(movies[['movieId', 'title', 'primary_genre']], on='movieId')
    print(recs[['title', 'primary_genre', 'similarity']].to_string(index=False))
else:
    print('Movie ID not found in embedded items.')

                                           title primary_genre  similarity
                  Godfather: Part II, The (1974)         Crime    0.886037
                               Fight Club (1999)        Action    0.855171
                Silence of the Lambs, The (1991)         Crime    0.776816
                                   Psycho (1960)         Crime    0.774969
                      Usual Suspects, The (1995)         Crime    0.762369
                               Goodfellas (1990)         Crime    0.762200
          One Flew Over the Cuckoo's Nest (1975)         Drama    0.745983
Pan's Labyrinth (Laberinto del fauno, El) (2006)         Drama    0.740167
                             12 Angry Men (1957)         Drama    0.738879
                            Departed, The (2006)         Crime    0.720165


In [4]:
# Movie journey: shortest similarity path between two titles
from src.models.journey import find_shortest_journey

START_ID = 858    # The Godfather
END_ID   = 50     # The Usual Suspects

path, cost = find_shortest_journey(journey_edges, START_ID, END_ID)
title_map = movies.set_index('movieId')['title'].to_dict()
if path:
    print(' -> '.join(title_map.get(m, str(m)) for m in path))
    print(f'Path length: {len(path)} | Cost: {cost:.4f}')
else:
    print('No path found.')

Godfather, The (1972) -> Usual Suspects, The (1995)
Path length: 2 | Cost: 0.2376


In [5]:
# Interactive preference network
from src.visualization.interactive_plots import build_plotly_preference_network

fig = build_plotly_preference_network(journey_edges, movies, max_nodes=100, dark_mode=False)
fig.show()

In [6]:
# Evaluate model metrics from saved JSON
import json
eval_path = MODELS / 'evaluation_summary.json'
if eval_path.exists():
    with open(eval_path) as f:
        ev = json.load(f)
    print('Prediction metrics:', ev.get('prediction_metrics'))
    print('Ranking metrics:   ', ev.get('ranking_metrics'))
    print('Coverage:          ', ev.get('coverage'))
    print('Diversity:         ', ev.get('diversity'))
else:
    print('evaluation_summary.json not found.')

Prediction metrics: {'rmse': 3.447627544403076, 'mae': 3.27237868309021}
Ranking metrics:    {'precision_at_k': 0.09, 'recall_at_k': 0.049410381285381284, 'users_evaluated': 20}
Coverage:           0.292
Diversity:          0.6168732393532992
